In [15]:
import sys
import os

path = os.path.abspath('../..')
if path not in sys.path:
    sys.path.insert(0, path)
    
sys.path

['/home/justin/Github/aqbuildtools',
 '/home/justin/Github/aqbuildtools/examples/2021RedesignCampaign',
 '/home/justin/anaconda3/envs/aqbt/lib/python38.zip',
 '/home/justin/anaconda3/envs/aqbt/lib/python3.8',
 '/home/justin/anaconda3/envs/aqbt/lib/python3.8/lib-dynload',
 '',
 '/home/justin/anaconda3/envs/aqbt/lib/python3.8/site-packages',
 '/home/justin/anaconda3/envs/aqbt/lib/python3.8/site-packages/IPython/extensions',
 '/home/justin/.ipython']

In [16]:
from aqbt import AquariumBuildTools
aqtools = AquariumBuildTools.from_toml('creds.secret.toml')
aqtools.sessions

aq = aqtools.sessions['production']['aquarium']
aq.set_timeout(70)

In [17]:
import networkx as nx

from bokeh.io import output_file, show
from bokeh.models import (BoxZoomTool, Circle, HoverTool,
                          MultiLine, Plot, Range1d, ResetTool, OpenURL, TapTool)
from bokeh.palettes import Spectral4
from bokeh.plotting import from_networkx
from bokeh.layouts import gridplot

from pydent import Planner
from pydent.models import Plan
from tqdm.auto import tqdm
from pydent import Planner
from matplotlib import pylab as plt
from pydent.planner.graph import PlannerLayout
from matplotlib.lines import Line2D


def resolve_plan(plan):
    if isinstance(plan, int):
        return aq.Plan.find(plan)
    elif isinstance(plan, Plan):
        return plan
    
def resolve_planner(planner):
    if isinstance(planner, (int, Plan)):
        return Planner(resolve_plan(planner))
    elif isinstance(planner, Planner):
        return planner
    
    
def create_legend():
    legend_elements = []
    for status, color in PlannerLayout.STATUS_COLORS.items():
        elm = Line2D([0], [0], color="w", marker='o', label=status, markerfacecolor=color, markersize=10, markeredgecolor='k')
        legend_elements.append(elm)
    return legend_elements

def plot_legend(ax, **kwargs):
    legend_elements = create_legend()
    ax.legend(handles=legend_elements)
    
    
def draw_plan(plan):
    planner = resolve_planner(plan)
    print(planner)
    print(planner.layout)
    g = planner.layout.nxgraph
    pos = planner.layout.pos()
    
    fig, ax = plt.subplots()
#     pos = nx.nx_agraph.graphviz_layout(g, prog='dot')
    nx.draw(g, pos=pos, node_size=200, node_color=planner.layout._status_colors(), linewidths=2, edgecolors='black', ax=ax)
    plot_legend(ax)
    plt.show()
    
def nxreverse(g):
    g2 = nx.DiGraph()
    for n, ndata in g.nodes(data=True):
        g2.add_node(n, **ndata)
    for n1, n2, edata in g.edges(data=True):
        g2.add_edge(n2, n1, **edata)
    return g2

def pprint_plan(plan, draw=True, save=False):
    planner = resolve_planner(plan)
    g = planner.layout.nxgraph
    pos = nx.nx_agraph.graphviz_layout(g, prog='dot')
    
    for op in planner.operations:
        x, y = pos[op.id]
        op.x = x * 2
        op.y = y
    print(planner.layout)
    if draw:
        draw_plan(planner)
    if save:
        planner.save()
        print(planner.url)
    return planner
    
def sid(sample):
    return '{}: {}'.format(sample.id, sample.name)
    
def get_output_samples(op):
    samples = []
    for fv in op.outputs:
        if fv.sample:
            samples.append(sid(fv.sample))
    return samples

def get_input_samples(op):
    samples = []
    for fv in op.inputs:
        if fv.sample:
            samples.append(sid(fv.sample))
    return samples
            
def interactive_plan_plot(plan):
    planner = resolve_planner(plan)

    g = planner.layout.nxgraph
    
    # positioning
    pos = planner.layout.pos()
    xpos = [xy[0] for xy in pos.values()]
    ypos = [xy[1] for xy in pos.values()]
    X_SPACER = 100
    Y_SPACER = 100

    # create new networkx
    G = nx.DiGraph()
    for node, ndata in g.nodes(data=True):
        op = ndata['operation']
        data = {
           'optype': op.operation_type.name,
            'status': op.status,
            'id': op.id,
            'input_samples': get_input_samples(op),
            'output_samples': get_output_samples(op),
            'color': planner.layout.STATUS_COLORS.get(op.status, 'k')
        }
        G.add_node(node, **data)
    for edge in g.edges:
        G.add_edge(*edge, edge_color='k')


    # plotting and rendering
    plot = Plot(plot_width=400, plot_height=400,
                x_range=Range1d(min(xpos) - X_SPACER, max(xpos) + X_SPACER), y_range=Range1d(min(ypos) - Y_SPACER, max(ypos) + Y_SPACER))
    plot.title.text = "Plan {}\n{}".format(planner.plan.id, planner.plan.name)

    node_hover_tool = HoverTool(tooltips=[("id", "@id"), ("type", "@optype"), ('status', '@status'), ('outputs', '@output_samples'), ('inputs', '@input_samples')])
    plot.add_tools(node_hover_tool, BoxZoomTool(), ResetTool(), TapTool(callback=OpenURL(url=planner.url)))

    graph_renderer = from_networkx(G, pos, scale=1, center=(0, 0))

    graph_renderer.node_renderer.glyph = Circle(size=15, fill_color='color')
    graph_renderer.edge_renderer.glyph = MultiLine(line_color="edge_color", line_alpha=0.8, line_width=1)
    plot.renderers.append(graph_renderer)

    return plot

def interactive_plan_grid(plans):
    if isinstance(plans, str):
        plans = aq.Plan.where({'folder': plans})
    
    plots = []
    for p in tqdm(plans):
        plot = interactive_plan_plot(p)
        plots.append(plot)

    plot = gridplot(plots, ncols=6)
    show(plot)

In [ ]:
# pprint_plan(plan_iddd, save=True)

In [18]:
interactive_plan_grid('SD2 CRISPR Redesign 2021')

  0%|          | 0/57 [00:00<?, ?it/s]

ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name. This could either be due to a misspelling or typo, or due to an expected column being missing. : key "line_color" value "edge_color" [renderer: GlyphRenderer(id=16712, glyph=MultiLine(id='16724', ...), ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name. This could either be due to a misspelling or typo, or due to an expected column being missing. : key "line_color" value "edge_color" [renderer: GlyphRenderer(id=16790, glyph=MultiLine(id='16802', ...), ...)]
